In [12]:
#build corpus
def DNA2Sentence(dna, K):

    sentence = ""
    length = len(dna)

    for i in range(length - K + 1):
        sentence += dna[i: i + K] + " "

    #delete extra space
    sentence = sentence[0 : len(sentence) - 1]
    return sentence

def Get_Unsupervised(fname,gname,kmer):
	f = open(fname,'r')
	g = open(gname,'w')
	k = kmer
	for i in f:
		if '>' not in i:
			i = i.strip('\n').upper()
			line = DNA2Sentence(i,k)
			g.write(line+'\n')
	f.close()
	g.close()

Get_Unsupervised('pos.fasta','pos2Un',2)#postive samples contained in 'pos.fasta';'pos2Un' is outputFile of corpus;'2' is the size of Kmer
Get_Unsupervised('neg.fasta','neg2Un',2)#negative samples contained in 'neg.fasta';'neg2Un' is outputFile of corpus;'2' is the size of Kmer

#combine two corpus and generate final corpus
with open('2Un','ab') as f:
        f.write(open('pos2Un','rb').read())
        f.write(open('neg2Un','rb').read())
        

In [13]:
#get model
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import pandas as pd
import numpy as np
import os
import sys
import math
import random
import warnings
from sklearn import preprocessing
import sklearn.preprocessing
from gensim import corpora, models, similarities

def getWord_model(word,num_features,min_count):
	word_model = ""
	if not os.path.isfile("model_2"):
		sentence = LineSentence("2Un",max_sentence_length = 15000)
		print ("Start Training Word2Vec model...")
		# Set values for various parameters
		num_features = int(num_features)	  # Word vector dimensionality
		min_word_count = int(min_count)	  # Minimum word count
		num_workers = 20		 # Number of threads to run in parallel并行运行的线程数
		context = 20			# Context window size上下文窗口大小
		downsampling = 1e-3	 # Downsample setting for frequent words常用词的下采样设置

		# Initialize and train the model初始化和训练模型
		print ("Training Word2Vec model...")
		word_model = Word2Vec(sentence, workers=num_workers,\
						size=num_features, min_count=min_word_count, \
						window=context, sample=downsampling, seed=1,iter = 50)
		word_model.init_sims(replace=False)
		word_model.save("model_2")
		#print word_model.most_similar("CATAGT")
	else:
		print ("Loading Word2Vec model...")
		word_model = Word2Vec.load("model_2")
		#word_model.init_sims(replace=True)
	return word_model

getWord_model(2,200,1)

Start Training Word2Vec model...
Training Word2Vec model...


In [14]:
#combine train data
def combine(posfile, negfile, allfile):
    f1 = open(posfile)
    f2 = open(negfile)
    g = open(allfile,'w')
    g.write('lable\tseq\n')
    for i in f1:
        if '>'not in i:
            g.write('1\t'+i)
    for i in f2:
        if '>'not in i:
            g.write('0\t'+i)
    f1.close()
    f2.close()
    g.close()

combine('pos.fasta','neg.fasta','all.fasta')#'pos.fasta' contains positive samples with fasta format; 'neg.fasta' contains negative samples with fasta format; 'all.fasta' is a combination file of positive and negative samples.

In [15]:
#obtain feature file with .npy format
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import pandas as pd
import numpy as np
import os
import sys
import math
import random

import warnings
from sklearn import preprocessing
import sklearn.preprocessing

def getDNA_split(DNAdata,word):
	DNAlist1 = []
	#DNAlist2 = []
	counter = 0
	for DNA in DNAdata["seq"]:
		#if counter % 100 == 0:
			#print ("DNA %d of %d\r" % (counter, 2*len(DNAdata)))
			#sys.stdout.flush()

		DNA = str(DNA).upper()
		DNAlist1.append(DNA2Sentence(DNA,word).split(" "))#[['ACG', 'CGT', 'GTC'],['ACG', 'CGT', 'GTC'],['ACG', 'CGT', 'GTC']]

		counter += 1
	return DNAlist1

#	print()
#	return DNAlist1,DNAlist2

#def getAvgFeatureVecs(DNAdata1,DNAdata2,model,num_features):
def getAvgFeatureVecs(DNAdata1,model,num_features):
	counter = 0
	DNAFeatureVecs = np.zeros((len(DNAdata1),num_features), dtype="float32")
	for DNA in DNAdata1:
		if counter % 1000 == 0:
			print ("DNA %d of %d\r" % (counter, len(DNAdata1)))
			sys.stdout.flush()

		DNAFeatureVecs[counter][0:num_features] = np.mean(model[DNA],axis = 0)
		counter += 1
	print()
	counter = 0
	return DNAFeatureVecs

def DNA2Sentence(dna, K):
	sentence = ""
	length = len(dna)

	for i in range(length - K + 1):
		sentence += dna[i: i + K] + " "

	#delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence

data = pd.read_csv('all.fasta',sep = "\t",error_bad_lines=False)
#datawords1,datawords2 = getDNA_split(data,4)
datawords1 = getDNA_split(data,2)#size of kmer
#print(datawords1)

word_model = Word2Vec.load("model_2")#'model_2' is above generated in 'get model'
#dataDataVecs = getAvgFeatureVecs(datawords1,datawords2,word_model,100)
dataDataVecs = getAvgFeatureVecs(datawords1,word_model,200)
print (dataDataVecs.shape)
np.save("2_vecs.npy",dataDataVecs)#outputFile

DNA 0 of 13314
DNA 1000 of 13314


D:\anaconda\condaSetup\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


DNA 2000 of 13314
DNA 3000 of 13314
DNA 4000 of 13314
DNA 5000 of 13314
DNA 6000 of 13314
DNA 7000 of 13314
DNA 8000 of 13314
DNA 9000 of 13314
DNA 10000 of 13314
DNA 11000 of 13314
DNA 12000 of 13314
DNA 13000 of 13314

(13314, 200)


In [17]:
#convert file format (.npy to .txt/.csv)
import pandas as pd
import numpy as np
import os
import sys
import math
import random

def npyToTXT(npyfile, svmfile, pos_num):
    dataDataVecs = np.load(npyfile)
    g = open(svmfile,'w')
    print(len(dataDataVecs))
    #print(dataDataVecs[0])

    m = 0
    for i in range(len(dataDataVecs)):
        line = ''
        for j in range(len(dataDataVecs[0])):
            if j == len(dataDataVecs[0])-1:
                line += str(j+1)+':'+str(dataDataVecs[i][j])+'\n'
            else:
                line += str(j+1)+':'+str(dataDataVecs[i][j])+'\t'
        m += 1
        if m < (pos_num+1):
            g.write('1\t'+line)
        else:
            g.write('0\t'+line)
            
def TXTtoCSV(svmfile, csvfile):
	f = open(svmfile,'r')
	g = open(csvfile,'w')
	lines = f.readlines()
	legth = len(lines[0].split('	'))-1
	#print(legth)
	classline = 'class'
	for i in range(legth):
		classline += ',%d'%(i+1)
	g.write(classline+'\n')

	for line in lines:
		line = line.strip('\n').split('	')
		g.write(line[0]+',')

		legth2 = len(line[1:])
		m = 0
		for j in line[1:]:
			if m == legth2-1:
				j = j.split(':')[-1]
				g.write(j)
				m += 1
			else:
				j = j.split(':')[-1]
				g.write(j+',')
				m += 1
		g.write('\n')

	f.close()
	g.close()
    
npyToTXT("2_vecs.npy", '2_vecs.txt',6657)#positive number
TXTtoCSV('2_vecs.txt', '2_vecs.csv')

13314
